In [5]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [6]:
def compute(delta_eta, eta_max, Nf, IC = "NewBorn", alpha = 0.25):
    # IC = initial condition
    
    Nc = 3
    N_ratio = Nf / Nc
    n_steps = int(eta_max / delta_eta)
    
    if IC == "Ones": # setting initial condition to all 1s 
        G = np.ones((n_steps + 1, n_steps + 1))
        G2 = np.ones((n_steps + 1, n_steps + 1))
        Q = np.ones((n_steps + 1, n_steps + 1))
        Gm = np.ones((n_steps + 1, 2*n_steps + 1, n_steps + 1))
        Gm2 = np.ones((n_steps + 1, 2*n_steps + 1, n_steps + 1))
        Gb = np.ones((n_steps + 1, 2*n_steps + 1, n_steps + 1))
         
    elif IC == "Born": # setting initial conditions to Born expression
        CF = ((Nc ** 2) - 1) / (2 * Nc)
        coef = ((alpha ** 2) * CF / (2 * Nc)) * np.pi * np.sqrt(2 * np.pi / (alpha * Nc)) * delta_eta
        G = np.array([[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gm = np.array([[[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])
        Q = np.array([[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gb = np.array([[[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])
        G2 = np.array([[coef * (i - n_steps+j) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gm2 = np.array([[[coef * (i - n_steps+j) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])
        
    else: # setting initial conditions to ??
        CF = ((Nc ** 2) - 1) / (2 * Nc)
        coef = ((alpha ** 2) * CF / (2 * Nc)) * np.pi * np.sqrt(2 * np.pi / (alpha * Nc)) * delta_eta
        G = np.array([[coef * ((CF * (j + n_steps)) - (2 * min(- i + n_steps, j))) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gm = np.array([[[coef * ((CF * (j + n_steps)) - (2 * min(- i + n_steps, j))) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])
        Q = np.array([[coef * ((CF * (j + n_steps)) - (2 * min(- i + n_steps, j))) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gb = np.array([[[coef * ((CF * (j + n_steps)) - (2 * min(- i + n_steps, j))) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])
        G2 = np.array([[coef * (i - n_steps+j) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gm2 = np.array([[[coef * (i - n_steps+j) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])

    
    for j in range(1, n_steps + 1): 
        for i in range(j - n_steps, j):     # For i=j, each amplitude just equals the IC. For Gammas, we have i=k=j.
            
            # Increment Q
            Q[i+n_steps-j, j] = Q[i+n_steps-j+1, j-1] 
            if IC == "Born" or IC == "NewBorn":
                Q[i+n_steps-j, j] += ((CF - 2) * coef)
            for ii in range(i-1, j-1):
                Q[i+n_steps-j, j] += (delta_eta ** 2) * (Q[ii+n_steps-j+1, j-1] + 2*G2[ii+n_steps-j+1, j-1]) / 2
            for ii in range(i, j-1):
                Q[i+n_steps-j, j] += (delta_eta ** 2) * (2*Gm[i+n_steps-j+1, ii+n_steps, j-1] + 2*G[ii+n_steps-j+1, j-1] + Q[ii+n_steps-j+1, j-1] - Gb[i+n_steps-j+1, ii+n_steps, j-1] + 2 * Gm2[i+n_steps-j+1, ii+n_steps, j-1] + 2 * G2[ii+n_steps-j+1, j-1])
            for jj in range(j-1):
                Q[i+n_steps-j, j] += (delta_eta ** 2) * (Q[i-j+n_steps, jj] + 2*G2[i-j+n_steps, jj]) / 2
                
            # Increment Gb (Gamma bar)
            Gb[i+n_steps-j, i+n_steps, j] = Q[i+n_steps-j, j]
            for k in range(i+1, j+1):
                Gb[i+n_steps-j, k+n_steps, j] = Gb[i+n_steps-j+1, k-1+n_steps, j-1]
                if IC == "Born" or IC == "NewBorn":
                    Gb[i+n_steps-j, k+n_steps, j] += ((CF - 2) * coef)
                for ii in range(k-1, j-1):
                    Gb[i+n_steps-j, k+n_steps, j] += (delta_eta ** 2) * (Q[ii+n_steps-j+1, j-1] + 2*G2[ii+n_steps-j+1, j-1]) / 2
                for ii in range(max(i, k-1), j-1):
                    Gb[i+n_steps-j, k+n_steps, j] += (delta_eta ** 2) * (2*Gm[i+n_steps-j+1, ii+n_steps, j-1] + 2*G[ii+n_steps-j+1, j-1] + Q[ii+n_steps-j+1, j-1] - Gb[i+n_steps-j+1, ii+n_steps, j-1] + 2 * Gm2[i+n_steps-j+1, ii+n_steps, j-1] + 2 * G2[ii+n_steps-j+1, j-1])
            
            # Increment G (G adjoint)
            G[i+n_steps-j, j] = G[i+n_steps-j+1, j-1] 
            if IC == "Born" or IC == "NewBorn":
                G[i+n_steps-j, j] += ((CF - 2) * coef)
            for jj in range(j-1):
                G[i+n_steps-j, j] += - (delta_eta ** 2) * (Q[i-j+n_steps, jj] + 2*G2[i-j+n_steps, jj]) * N_ratio / 4
            for ii in range(i, j-1):
                G[i+n_steps-j, j] += (delta_eta ** 2) * (Gm[i+n_steps-j+1, ii+n_steps, j-1] + 3 * G[ii+n_steps-j+1, j-1] + 2 * G2[ii+n_steps-j+1, j-1] + ((4 - N_ratio) / 2) * Gm2[i+n_steps-j+1, ii+n_steps, j-1] - (N_ratio / 4) * Gb[i+n_steps-j+1, ii+n_steps, j-1])
            G[i+n_steps-j, j] += - (delta_eta ** 2) * (Q[i-1+n_steps-j+1, j-1] + 2*G2[i-1+n_steps-j+1, j-1]) * N_ratio / 4
                
            # Increment Gm (Gamma adjoint)
            Gm[i+n_steps-j, i+n_steps, j] = G[i+n_steps-j, j]
            for k in range(i+1, j+1):
                Gm[i+n_steps-j, k+n_steps, j] = Gm[i+n_steps-j+1, k-1+n_steps, j-1]
                if IC == "Born" or IC == "NewBorn":
                    Gm[i+n_steps-j, k+n_steps, j] += ((CF - 2) * coef)
                for ii in range(max(i, k-1), j-1):
                    Gm[i+n_steps-j, k+n_steps, j] += (delta_eta ** 2) * (Gm[i+n_steps-j+1, ii+n_steps, j-1] + 3 * G[ii+n_steps-j+1, j-1] + 2 * G2[ii+n_steps-j+1, j-1] + ((4 - N_ratio) / 2) * Gm2[i+n_steps-j+1, ii+n_steps, j-1] - (N_ratio / 4) * Gb[i+n_steps-j+1, ii+n_steps, j-1])
            
            # Increment G2 (G2 adjoint)
            G2[i+n_steps-j, j] = G2[i+n_steps-j+1, j-1]
            for jj in range(j-1):
                G2[i+n_steps-j, j] += 2 * (delta_eta ** 2) * (G[i-j+n_steps, jj] + 2*G2[i-j+n_steps, jj])
            G2[i+n_steps-j, j] += 2 * (delta_eta ** 2) * (G[i-1+n_steps-j+1, j-1] + 2*G2[i-1+n_steps-j+1, j-1])
            
            # Increment Gm2 (Gamma 2 adjoint)
            Gm2[i+n_steps-j, i+n_steps, j] = G2[i+n_steps-j, j]
            for k in range(i+1, j+1):
                Gm2[i+n_steps-j, k+n_steps, j] = Gm2[i+n_steps-j+1, k-1+n_steps, j-1]
                
        print(np.round(j * delta_eta, 4))
        
    return Q, G, G2

### Delta = 0.05

In [7]:
Q, G, G2 = compute(0.05, 40, 3, IC = "Ones")

0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
1.0
1.05
1.1


KeyboardInterrupt: 

In [4]:
np.savetxt("Q00540Nf3_new_Ones.dat", Q)

NameError: name 'Q' is not defined

In [ ]:
np.savetxt("G00540Nf3_new_Ones.dat", G)

In [ ]:
np.savetxt("GG00540Nf3_new_Ones.dat", G2)